### Repetir tarifas mensuales por cada mes hasta obtener lista con 8760 datos

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

# Solicita la tarifa mensual al usuario
tarifas_mensuales = [
    0.1225,
    0.1820,
    0.1530,
    0.1320,
    0.1350,
    0.1210,
    0.1550,
    0.1560,
    0.1630,
    0.1710,
    0.1715,
    0.1875
]

trm_dolar = 4200


# Genera la serie horaria para el año completo
horas = []
tarifas_horarias = []

# Fecha inicial
fecha = datetime(2024, 1, 1, 0, 0)  # Puedes ajustar el año si es necesario

# Genera las horas y la tarifa correspondiente
for mes in range(12):
    # Calcula el número de días en el mes
    if mes + 1 in [1, 3, 5, 7, 8, 10, 12]:
        dias_mes = 31
    elif mes + 1 in [4, 6, 9, 11]:
        dias_mes = 30
    else:
        # Febrero y años bisiestos (2024 es bisiesto)
        dias_mes = 29 if fecha.year % 4 == 0 else 28

    # Agrega la tarifa horaria para cada hora del mes
    for _ in range(dias_mes * 24):
        horas.append(fecha)
        tarifas_horarias.append(tarifas_mensuales[mes])
        fecha += timedelta(hours=1)

# Crea un DataFrame con los resultados
df = pd.DataFrame({'FechaHora': horas, 'Tarifa': np.round(tarifas_horarias,4)})

# Exporta a un archivo Excel
df.to_excel('01_exportacion/tarifa_horaria.xlsx', index=False)
print("Archivo 'tarifa_horaria.xlsx' creado exitosamente.")

### Generar perfil de indisponibilidad red gas propano

In [ ]:
import numpy as np
import random
import pandas as pd

# Total de horas en el año
total_hours = 8760
# Total de horas de indisponibilidad (1 mes)
unavailable_hours = 720
# Lista de disponibilidad inicializada con 1 (disponible)
availability_profile = np.ones(total_hours, dtype=int)

# Configuración de la longitud mínima de cada franja de indisponibilidad
min_unavailability_block = 5

# Distribuir las horas de indisponibilidad en franjas
remaining_unavailable_hours = unavailable_hours
while remaining_unavailable_hours > 0:
    # Definir el tamaño de la siguiente franja
    block_size = random.randint(min_unavailability_block, min(min_unavailability_block * 3, remaining_unavailable_hours))
    
    # Encontrar una posición aleatoria para insertar la franja de indisponibilidad
    start_pos = random.randint(0, total_hours - block_size)
    
    # Asegurarse de que no se solapen las franjas de indisponibilidad
    if np.all(availability_profile[start_pos:start_pos + block_size] == 1):
        # Insertar la franja de indisponibilidad
        availability_profile[start_pos:start_pos + block_size] = 0
        # Restar las horas de indisponibilidad añadidas
        remaining_unavailable_hours -= block_size

# Crear un DataFrame con la serie de disponibilidad
df = pd.DataFrame({
    "Hora": range(1, total_hours + 1),    # Columna de horas
    "Disponibilidad": availability_profile # Columna de disponibilidad (0 o 1)
})

# Exportar a un archivo Excel
df.to_excel("01_exportacion/perfil_disponibilidad_gas_propano.xlsx", index=False)

print("Perfil de disponibilidad exportado exitosamente a 'perfil_disponibilidad_gas_propano.xlsx'")

### Carga promedio diaria por mes a perfil de 8760 datos donde cada día se desvía un porcentaje dado

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Diccionario de cargas promedio diarias para cada mes
carga = pd.read_excel("CaseData.xlsx", sheet_name= "carga_mes_rosalia").to_dict(orient="list")

# Orden de los meses
meses = ["enero", "febrero", "marzo", "abril", "mayo", "junio", "julio", "agosto", "septiembre", "octubre", "noviembre", "diciembre"]

# Lista para almacenar la serie horaria
horas = []
cargas_horarias = []

# Fecha inicial
fecha = datetime(2019, 1, 1, 0, 0)  # Puedes ajustar el año si es necesario

# Generación de la serie horaria
for mes in meses:
    dias_mes = 31 if mes in ["enero", "marzo", "mayo", "julio", "agosto", "octubre", "diciembre"] else 30
    if mes == "febrero":
        dias_mes = 29 if fecha.year % 4 == 0 else 28

    # Genera las cargas horarias con variación aleatoria del 15%
    for dia in range(dias_mes):
        for hora in range(24):
            # Carga promedio del día con variación aleatoria
            carga_base = carga[mes][hora]
            variacion = random.uniform(-0.15, 0.15)  # Variación aleatoria entre -15% y +15%
            carga_horaria = carga_base * (1 + variacion)

            # Agrega a las listas de horas y cargas
            horas.append(fecha)
            cargas_horarias.append(carga_horaria)
            fecha += timedelta(hours=1)

# Crear DataFrame con resultados
df = pd.DataFrame({'FechaHora': horas, 'Carga_kW': np.round(cargas_horarias,4)})

# Exportar a Excel
df.to_excel('01_exportacion/carga_horaria_8760.xlsx', index=False)
print("Archivo 'carga_horaria_8760.xlsx' creado exitosamente.")


Archivo 'carga_horaria_8760.xlsx' creado exitosamente.


### Crear perfil de indisponibilidad red eléctrica a partir de cantida de horas mensuales de indisponibilidad del IPSE

In [3]:
import pandas as pd
import numpy as np
import math

def distribuir_indisponibilidad(horas_indisponibles, total_horas=24):
    """
    Distribuye las horas de indisponibilidad en franjas continuas para un día.
    - Si horas_indisponibles es 0, todo el día está disponible.
    - Si horas_indisponibles es 1, la hora no disponible se coloca aleatoriamente.
    - Para otras cantidades, las franjas se distribuyen evitando bloques de 1 hora sueltos.
    """
    if horas_indisponibles == 0:
        return [1] * total_horas  # Todo el día disponible
    elif horas_indisponibles == 1:
        disponibilidad = [1] * total_horas
        indisponible_idx = np.random.randint(0, total_horas)  # Seleccionar una hora aleatoria
        disponibilidad[indisponible_idx] = 0
        return disponibilidad
    else:
        horas_disponibles = total_horas - horas_indisponibles
        disponibilidad = [1] * horas_disponibles + [0] * horas_indisponibles
        np.random.shuffle(disponibilidad)
        while True:
            bloques = "".join(map(str, disponibilidad)).split("1")
            if all(len(b) != 1 for b in bloques if len(b) != horas_indisponibles):
                break
            np.random.shuffle(disponibilidad)
    return disponibilidad

def generar_serie_binaria(df):
    """
    Genera una serie binaria horaria con base en una tabla de fechas e indisponibilidades diarias.
    """
    serie_binaria = []
    for _, row in df.iterrows():
        fecha = row['Fecha']
        indisponibilidad = math.ceil(row['Indisponibilidad'])  # Aproximar al entero superior
        serie_binaria += distribuir_indisponibilidad(indisponibilidad)
    return serie_binaria

# Leer datos desde Excel
df = pd.read_excel("CaseData.xlsx", sheet_name= "indisponibilidad_rosalia")

# Generar serie binaria horaria
serie_horaria = generar_serie_binaria(df)

# Validar que la serie tiene 8760 datos
assert len(serie_horaria) == 8760, "La serie generada no tiene 8760 datos."

# Exportar a un archivo si es necesario
output = pd.DataFrame({'hora': range(1, 8761), 'estado': serie_horaria})
output.to_excel("01_exportacion/serie_binaria_red.xlsx", index=False)

print("Serie binaria generada y exportada exitosamente.")




Serie binaria generada y exportada exitosamente.
